# Importación de librerías

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from time import time

%matplotlib inline
%matplotlib notebook
plt.style.use('default')

# Operaciones con los dataframes

### DF 1: Educación de los postulantes

Realizamos un merge de los 3 dataframes de educación de los postulantes que nos fueron provistos, para poder trabajar de manera unificada.

In [2]:
postulantes_educ = pd.read_csv('datos_navent_fiuba/h15_fiuba_1_postulantes_educacion.csv')
postulantes_educ_2 = pd.read_csv('datos_navent_fiuba/d15_fiuba_1_postulantes_educacion.csv')

postulantes_educ = pd.merge(postulantes_educ, postulantes_educ_2, on=['idpostulante', 'nombre', 'estado']\
                       , how='outer')

postulantes_educ_2 = pd.read_csv('datos_navent_fiuba/fiuba_1_postulantes_educacion.csv')

postulantes_educ = pd.merge(postulantes_educ, postulantes_educ_2, on=['idpostulante', 'nombre', 'estado']\
                       , how='outer')

# Renombramos la columna 'nombre' por algo mas apropiado
postulantes_educ.rename(columns={'nombre':'titulo_univ'}, inplace=True)

postulantes_educ.head()

,idpostulante,titulo_univ,estado
0,ZjlZ,Master,En Curso
1,NdJl,Posgrado,En Curso
2,5kNq,Otro,En Curso
3,8rYD,Master,En Curso
4,1Wvj,Universitario,En Curso


In [3]:
postulantes_educ.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 693641 entries, 0 to 693640
Data columns (total 3 columns):
idpostulante    693641 non-null object
titulo_univ     693641 non-null object
estado          693641 non-null object
dtypes: object(3)
memory usage: 21.2+ MB


Creamos una nueva columna que indica si el postulante en cuestión tiene un título en curso o ya se graduó/ abandonó

In [4]:
postulantes_educ['estado'].value_counts()

Graduado      451142
En Curso      182338
Abandonado     60161
Name: estado, dtype: int64

In [5]:
postulantes_educ.loc[(postulantes_educ['estado'] == 'Graduado'), 'esta_estudiando'] = 0
postulantes_educ.loc[(postulantes_educ['estado'] == 'Abandonado'), 'esta_estudiando'] = 0
postulantes_educ.loc[(postulantes_educ['estado'] == 'En Curso'), 'esta_estudiando'] = 1

postulantes_educ.sample(5)

,idpostulante,titulo_univ,estado,esta_estudiando
99855,Yw85JJ,Universitario,Graduado,0.0
680865,5mP5OzM,Secundario,Graduado,0.0
482498,jkjqAje,Universitario,En Curso,1.0
282622,3NzxD66,Secundario,Graduado,0.0
4540,BmBvz6E,Terciario/Técnico,En Curso,1.0


Transformamos los strings de los títulos universitarios en variables numéricas.
En principio esto lo realizamos para poder eliminar los IDs duplicados del dataframe. Aquí se contemplan los casos
de postulantes que ingresaron con distintos títulos o que el estado de su título varió entre una postulación
y la otra. A su vez, esto nos servirá más adelante para aplicar los algoritmos de Machine Learning.

Analizamos los distintos tipos de títulos universitarios que existen.

In [6]:
postulantes_educ.groupby('titulo_univ')['titulo_univ'].count()

titulo_univ
Doctorado               620
Master                10074
Otro                  53363
Posgrado              20624
Secundario           244689
Terciario/Técnico    113421
Universitario        250850
Name: titulo_univ, dtype: int64

Pasamos a variables numéricas cada una de las combinaciones de título y estado de título, a fin de armar un orden de jerarquía de títulos, de manera tal de quedarnos por cada ID del postulante con su título más avanzado.

NOTA: consideramos que 'Otro' es el nivel jerárquico más bajo.

In [7]:
# Otro
postulantes_educ.loc[(postulantes_educ["titulo_univ"] == "Otro"), 'estudios'] = 1
 
# Secundario
postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Secundario'), 'estudios'] = 2

# Terciario/ Técnico
postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Terciario/Técnico'), 'estudios'] = 3

# Universitario
postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Universitario'), 'estudios'] = 4

# Posgrado
postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Posgrado'), 'estudios'] = 5

# Master
postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Master'), 'estudios'] = 6

# Doctorado
postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Doctorado'), 'estudios'] = 7

postulantes_educ.sample(5)

,idpostulante,titulo_univ,estado,esta_estudiando,estudios
639826,EzlBe0z,Secundario,Graduado,0.0,2.0
344326,53zVqk,Terciario/Técnico,Graduado,0.0,3.0
104680,KBrVaNX,Terciario/Técnico,Graduado,0.0,3.0
193822,6rZxpxr,Universitario,En Curso,1.0,4.0
146865,0zPYLZr,Secundario,Graduado,0.0,2.0


Analizamos la cantidad de IDs duplicados

In [8]:
IDs_duplicados = sum(postulantes_educ['idpostulante'].value_counts() > 1)
IDs_duplicados

180088

Ordenamos el dataframe según el orden jerárquico de estudio en forma descendente.

In [9]:
postulantes_educ = postulantes_educ.sort_values('estudios', ascending=False)

Eliminamos todos los IDs duplicados. Considerando que previamente se ordenó el dataframe por orden de estudio, quedará un único ID por postulante con su título más avanzado.

In [10]:
postulantes_educ = postulantes_educ.drop_duplicates('idpostulante')
postulantes_educ.sample(5)

,idpostulante,titulo_univ,estado,esta_estudiando,estudios
393331,3NPLL2x,Universitario,Abandonado,0.0,4.0
64888,149A1B,Universitario,Graduado,0.0,4.0
10967,KBzrA3A,Universitario,Graduado,0.0,4.0
251533,4reExV0,Universitario,En Curso,1.0,4.0
266547,96M0Gbv,Secundario,Graduado,0.0,2.0


Comprobamos que efectivamente no quedaron IDs duplicados.

In [11]:
IDs_duplicados = sum(postulantes_educ['idpostulante'].value_counts() > 1)
IDs_duplicados

0

Eliminamos las columnas 'titulo_univ' y 'estado', ya que la información relativa a esto ya se encuentra contenida en 'orden_estudio' y en 'esta_estudiando'

In [12]:
postulantes_educ.drop(columns={'titulo_univ', 'estado'}, axis=1, inplace=True)
postulantes_educ.sample(5)

,idpostulante,esta_estudiando,estudios
589636,ZKXwAb,0.0,4.0
447149,owaeJRk,1.0,4.0
477805,eXVvX4,0.0,4.0
681751,0zPM549,1.0,2.0
129248,ow28Ka3,0.0,2.0


In [13]:
postulantes_educ.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 447909 entries, 393408 to 530578
Data columns (total 3 columns):
idpostulante       447909 non-null object
esta_estudiando    447909 non-null float64
estudios           447909 non-null float64
dtypes: float64(2), object(1)
memory usage: 13.7+ MB


### DF 2: Nacimiento y sexo de los postulantes

Realizamos un merge de los 3 dataframes de nacimiento y sexo de los postulantes que nos fueron provistos, para poder trabajar de manera unificada.

In [14]:
postulantes_gen_nac = pd.read_csv('datos_navent_fiuba/h15_fiuba_2_postulantes_genero_y_edad.csv')
postulantes_gen_nac_2 = pd.read_csv('datos_navent_fiuba/d15_fiuba_2_postulantes_genero_y_edad.csv')

postulantes_gen_nac = pd.merge(postulantes_gen_nac, postulantes_gen_nac_2, how='outer')

postulantes_gen_nac_2 = pd.read_csv('datos_navent_fiuba/fiuba_2_postulantes_genero_y_edad.csv')

postulantes_gen_nac = pd.merge(postulantes_gen_nac, postulantes_gen_nac_2, how='outer')

postulantes_gen_nac.head()

,idpostulante,fechanacimiento,sexo
0,6MM,1985-01-01,MASC
1,Nzz,NaN,NO_DECLARA
2,ZX1,NaN,NO_DECLARA
3,Nq5,NaN,NO_DECLARA
4,ebE,1952-07-07,MASC


In [15]:
postulantes_gen_nac.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 505382 entries, 0 to 505381
Data columns (total 3 columns):
idpostulante       505382 non-null object
fechanacimiento    478699 non-null object
sexo               505382 non-null object
dtypes: object(3)
memory usage: 15.4+ MB


Analizamos los posibles valores de la columna 'sexo'

In [16]:
postulantes_gen_nac['sexo'].value_counts()

FEM           251431
MASC          228008
NO_DECLARA     25936
0.0                7
Name: sexo, dtype: int64

Descartamos los registros que figuran con 'sexo' == 0.0, ya que son una pequeñísima parte del set de datos y no brindan información consistente.

In [17]:
postulantes_gen_nac = postulantes_gen_nac[postulantes_gen_nac['sexo'] != '0.0']

Comprobamos que efectivamente fueron eliminados esos registros

In [18]:
postulantes_gen_nac['sexo'].value_counts()

FEM           251431
MASC          228008
NO_DECLARA     25936
Name: sexo, dtype: int64

Analizamos la cantidad de IDs duplicados

In [19]:
IDs_duplicados = sum(postulantes_gen_nac['idpostulante'].value_counts() > 1)
IDs_duplicados

972

Ordenamos el dataframe según el sexo en forma ascendente.

In [20]:
postulantes_gen_nac = postulantes_gen_nac.sort_values('sexo', ascending=True)

Eliminamos todos los IDs duplicados. Considerando que previamente se ordenó el dataframe por sexo y que 'NO_DECLARA' tiene un orden alfabético menor que 'FEM' y 'MASC', quedará un único ID por postulante con su información más específica de sexo.

Interpretamos que la información concreta del sexo nos será relevante en posteriores análisis, por eso filtramos los IDs duplicados de esta forma.

In [21]:
postulantes_gen_nac = postulantes_gen_nac.drop_duplicates('idpostulante')
postulantes_gen_nac.sample(5)

,idpostulante,fechanacimiento,sexo
175582,1QdAvjb,1983-08-29,MASC
302897,NAKlDL,1981-05-26,MASC
316210,ak4p2G0,1977-12-22,FEM
202619,96zQq58,1983-09-18,FEM
445252,akDwKZ9,1994-09-21,FEM


Se transforma la columna 'fechanacimiento' en 'edad', mediante una serie de operaciones lógicas.

In [22]:
# Renombramos la columna por algo más apropiado
postulantes_gen_nac = postulantes_gen_nac.rename(columns={'fechanacimiento': 'edad'})

# Obtenemos el año de nacimiento
fecha_nac = postulantes_gen_nac['edad'].str.split('-')
año_nac = fecha_nac.str[0]
postulantes_gen_nac['edad'] = año_nac
postulantes_gen_nac['edad'] = pd.to_numeric(postulantes_gen_nac['edad'], errors='coerce').fillna(0).astype(np.int64)

# Filtramos edades inválidas
edad_min = postulantes_gen_nac['edad'] > 2000
edad_max = postulantes_gen_nac['edad'] < 1950
condicion_final = ((edad_min | edad_max))
postulantes_gen_nac = postulantes_gen_nac[np.logical_not(condicion_final)]

# Obtenemos la edad real según el año actual
postulantes_gen_nac['edad'] = postulantes_gen_nac['edad'].apply(lambda x: 2018-x)

postulantes_gen_nac.sample(5)

,idpostulante,edad,sexo
101802,owj5eva,50,FEM
124280,8MaplRD,33,MASC
502616,vVjdQKQ,20,FEM
192454,1QkYRKE,49,FEM
462629,GNdY9W9,21,FEM


Creamos la columna rango_edad para minimizar la cantidad de valores posibles correspondientes a la edad del postulante.

Rangos:
- 18-25
- 26-30
- 31-40
- 41-68

In [23]:
rango_18_25 = (postulantes_gen_nac['edad'] >= 18) & (postulantes_gen_nac['edad'] <= 25)
rango_26_30 = (postulantes_gen_nac['edad'] >= 26) & (postulantes_gen_nac['edad'] <= 30)
rango_31_40 = (postulantes_gen_nac['edad'] >= 31) & (postulantes_gen_nac['edad'] <= 40)
rango_41_68 = (postulantes_gen_nac['edad'] >= 41) & (postulantes_gen_nac['edad'] <= 68)

postulantes_gen_nac.loc[(rango_18_25), 'rango_edad'] = 1
postulantes_gen_nac.loc[(rango_26_30), 'rango_edad'] = 2
postulantes_gen_nac.loc[(rango_31_40), 'rango_edad'] = 3
postulantes_gen_nac.loc[(rango_41_68), 'rango_edad'] = 4

postulantes_gen_nac.sample(5)

,idpostulante,edad,sexo,rango_edad
340202,dYj3jvb,26,MASC,2.0
227030,NzrVx65,44,FEM,4.0
340869,Oqr3ZY6,27,FEM,2.0
35271,6avA5O,31,FEM,3.0
329174,jkM4vbY,24,MASC,1.0


Eliminamos la columna edad. Consideramos que la información importante se encuentra contenida en el rango edad.

In [24]:
postulantes_gen_nac.drop(columns={'edad'}, axis=1, inplace=True)
postulantes_gen_nac.sample(5)

,idpostulante,sexo,rango_edad
311634,LNed41r,FEM,1.0
297606,ow2M3BW,MASC,1.0
503602,KBrDJrp,FEM,2.0
40783,Nwz4rV,MASC,3.0
173381,NzPa1vx,FEM,2.0


In [25]:
postulantes_gen_nac.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 478001 entries, 165770 to 147639
Data columns (total 3 columns):
idpostulante    478001 non-null object
sexo            478001 non-null object
rango_edad      478001 non-null float64
dtypes: float64(1), object(2)
memory usage: 14.6+ MB


### Merge de los DF 1 y 2

Se realiza un merge de los dos dataframes con información correspondiente a los postulantes

In [26]:
postulantes = pd.merge(postulantes_educ, postulantes_gen_nac, on='idpostulante', how='inner')

postulantes.head()

,idpostulante,esta_estudiando,estudios,sexo,rango_edad
0,a0XaWD,0.0,7.0,MASC,4.0
1,4reGo5z,0.0,7.0,FEM,2.0
2,X9lpKkb,0.0,7.0,MASC,4.0
3,RzMXJ4E,0.0,7.0,FEM,4.0
4,akOA6b9,0.0,7.0,MASC,4.0


Pasamos a valor numérico el sexo de los postulantes.

- FEM = 1
- MASC = 2

In [27]:
postulantes.loc[(postulantes['sexo'] == 'FEM'),'sexo'] = 1

postulantes.loc[(postulantes['sexo'] == 'MASC'),'sexo'] = 2

postulantes.sample(5)

,idpostulante,esta_estudiando,estudios,sexo,rango_edad
203646,0zdJKJY,0.0,4.0,2,2.0
351986,1QPZa2B,0.0,2.0,2,3.0
118315,0zPOGJE,1.0,4.0,2,1.0
413301,VNxz1rN,0.0,2.0,2,1.0
227469,1ReQL3,1.0,4.0,1,2.0


### DF 3: Vistas de los avisos

Realizamos un merge de los 3 dataframes de vistas de los postulantes a los anuncios que nos fueron provistos, para poder trabajar de manera unificada.

In [28]:
vistas = pd.read_csv('datos_navent_fiuba/fiuba_3_vistas.csv')
vistas_2 = pd.read_csv('datos_navent_fiuba/d15_fiuba_3_vistas.csv')

vistas = pd.merge(vistas, vistas_2, how='outer')

vistas_2 = pd.read_csv('datos_navent_fiuba/h15_fiuba_3_vistas.csv')

vistas = pd.merge(vistas, vistas_2, how='outer')

# Renombramos algunas columnas por algo más apropiado
vistas.rename(columns={'idAviso':'idaviso'}, inplace=True)
vistas.rename(columns={'timestamp':'fechavista'}, inplace=True)

vistas.sample(5)

,idaviso,fechavista,idpostulante
8764821,1112489028,2018-04-25T10:17:44.581-0400,kPLl28a
8302654,1112463393,2018-04-17T00:19:24.368-0400,vVD8PxP
3769852,1804485,2018-04-18T07:52:10.218-0400,jeGkKY
16905272,1112223876,2018-04-03T11:58:23.291-0400,EK15aN
4920898,1112430337,2018-04-24T16:54:54.445-0400,YKW64q


In [29]:
vistas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18024382 entries, 0 to 18024381
Data columns (total 3 columns):
idaviso         int64
fechavista      object
idpostulante    object
dtypes: int64(1), object(2)
memory usage: 550.1+ MB


Eliminamos los registros en los cuales un mismo idpostulante visitó un mismo aviso en el mismo día. Esto es para
evitar usuarios que actualizaron muchas veces la página y casos similares.

Para realizar esto, previamente cambiamos el formato de la fecha.

In [30]:
# Modificamos el formato de la fecha vista
lista = vistas['fechavista'].str.split("T")
fecha = lista.str[0]
vistas['fechavista'] = fecha

# Eliminamos los registros arriba mencionados
vistas = vistas.drop_duplicates(['idaviso', 'fechavista', 'idpostulante'])
vistas.head()

MemoryError: 

Ordenamos el dataframe según la 'fechavista' en forma descendente, para tener las fechas más actuales en primer lugar. De esta forma, al generar la columna cant_vistas, nos quedaremos con la última vista del postulante al aviso.

In [ ]:
vistas = vistas.sort_values('fechavista', ascending=False)

Agrupamos por (idaviso, idpostulante) para quedarnos con un único registro por cada par y agregamos la columna cant_vistas, la cual determina cuántas veces un postulante vio un determinado aviso.

El registro que quedará será el más actual, según lo explicado arriba. Como ya tenemos la información que consideramos relevante en la columna 'cant_vistas', nos desentendemos de la columna 'fechavista', que creemos que no aportará información de peso para los algoritmos de Machine Learning.

In [ ]:
vistas = vistas.groupby(['idaviso','idpostulante']).size().to_frame('cant_vistas').reset_index()
vistas.sample(5)

In [ ]:
vistas.info()

### DF 4: Postulaciones a los avisos

Realizamos un merge de los 2 dataframes de postulaciones a los anuncios que nos fueron provistos, para poder trabajar de manera unificada.

In [ ]:
postulaciones = pd.read_csv('datos_navent_fiuba/fiuba_4_postulaciones.csv')
postulaciones_2 = pd.read_csv('datos_navent_fiuba/h15_fiuba_4_postulaciones.csv')

postulaciones = pd.merge(postulaciones, postulaciones_2, how='outer')

postulaciones.head()

In [ ]:
postulaciones.info()

### Merge de los DF 3 y 4

Se realiza un merge de los dos dataframes con información correspondiente a las vistas y postulaciones a los anuncios por parte de los postulantes.

In [ ]:
modelo = pd.merge(vistas, postulaciones, on=['idpostulante', 'idaviso'], how='outer')

modelo.head()

Eliminamos la columna 'fechapostulacion' y la transformamos a una columna binaria llamada 'sepostulo'.

De esta forma, aquellos registros que figuraban con una fecha efectiva en 'fechapostulacion', tendrán un 1 en la nueva columna. En cambio, aquellos registros que figuraban con NaN, tendrán un 0.

In [ ]:
modelo.loc[(pd.notna(modelo['fechapostulacion'])), 'fechapostulacion'] = 1   # Not NaN
modelo.loc[(modelo['fechapostulacion'].isnull()), 'fechapostulacion'] = 0    # NaN

modelo.rename(columns={'fechapostulacion':'sepostulo'}, inplace=True)

modelo["sepostulo"].value_counts()

In [ ]:
modelo.info()

Eliminamos los idaviso e idpostulante repetidos, para dejar un id único

In [ ]:
modelo = modelo.drop_duplicates(['idaviso','idpostulante'])
modelo.info()

### DF 6: Detalles de los avisos

Se realiza un merge de los 3 dataframes de detalles de los avisos que nos fueron provistos

In [ ]:
avisos_detalles = pd.read_csv('datos_navent_fiuba/h15_fiuba_6_avisos_detalle.csv')
avisos_detalles_2 = pd.read_csv('datos_navent_fiuba/d15_fiuba_6_avisos_detalle.csv')

avisos_detalles = pd.merge(avisos_detalles, avisos_detalles_2, how='outer')

avisos_detalles_2 = pd.read_csv('datos_navent_fiuba/fiuba_6_avisos_detalle.csv')

avisos_detalles = pd.merge(avisos_detalles, avisos_detalles_2, how='outer')

avisos_detalles.sample(5)

In [ ]:
avisos_detalles.info()

Eliminamos los avisos duplicados

In [ ]:
avisos_detalles = avisos_detalles.drop_duplicates(['idaviso'])

In [ ]:
# Renombramos la columna denominacion_empresa por algo más apropiado
avisos_detalles.rename(columns={'denominacion_empresa':'nombre_empresa'}, inplace=True)

Eliminamos las columnas que consideramos no relevantes. Según la explicación aquí abajo, éstas seran 'idpais', 'ciudad' y 'mapacalle'.

In [ ]:
# Analizamos la cantidad de 'idpais' posibles
# Dado que hay un único idpais, eliminaremos esta columna.
avisos_detalles['idpais'].value_counts()

In [ ]:
# Analizamos la cantidad de valores nulos en 'ciudad'
# Dado el gran porcentaje de valores nulos, eliminaremos la columna.
avisos_detalles['ciudad'].isnull().value_counts()

In [ ]:
# Analizamos la cantidad de valores nulos en 'mapacalle'
# Dado el gran porcentaje de valores nulos, eliminaremos la columna.
avisos_detalles['mapacalle'].isnull().value_counts()

In [ ]:
# Eliminamos las columnas arriba mencionadas
avisos_detalles.drop(columns={'idpais', 'ciudad', 'mapacalle'}, axis=1, inplace=True)

Pasamos a valor numérico el nombre zona, según un orden jerárquico asignado por nosotros. Entendemos que Gran Buenos Aires lidera el orden jerárquico, considerando la gran cantidad de registros con los que cuenta. Luego lo siguen Capital Federal y Otros (todas las zonas restantes)

In [ ]:
avisos_detalles['nombre_zona'].value_counts()

In [ ]:
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Gran Buenos Aires'),'nombre_zona'] = 3
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Capital Federal'),'nombre_zona'] = 2
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Buenos Aires (fuera de GBA)'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'GBA Oeste'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'La Plata'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Ciudad de Mendoza'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Mendoza'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Catamarca'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Rosario'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Santa Cruz'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'San Juan'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Cordoba'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Tucuman'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Neuquen'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Santa Fe'),'nombre_zona'] = 1

Pasamos a valor numérico el tipo de trabajo, según un orden jerárquico asignado por nosotros.

In [ ]:
avisos_detalles['tipo_de_trabajo'].value_counts()

In [ ]:
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Full-time'),'tipo_de_trabajo'] = 10
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Part-time'),'tipo_de_trabajo'] = 9
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Teletrabajo'),'tipo_de_trabajo'] = 8
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Por Horas'),'tipo_de_trabajo'] = 7
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Pasantia'),'tipo_de_trabajo'] = 6
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Temporario'),'tipo_de_trabajo'] = 5
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Por Contrato'),'tipo_de_trabajo'] = 4
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Fines de Semana'),'tipo_de_trabajo'] = 3
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Primer empleo'),'tipo_de_trabajo'] = 2
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Voluntario'),'tipo_de_trabajo'] = 1

Pasamos a valor numérico el nivel laboral, según un orden jerárquico asignado por nosotros.

In [ ]:
avisos_detalles['nivel_laboral'].value_counts()

In [ ]:
avisos_detalles.loc[(avisos_detalles['nivel_laboral'] == 'Gerencia / Alta Gerencia / Dirección'),'nivel_laboral'] = 5
avisos_detalles.loc[(avisos_detalles['nivel_laboral'] == 'Jefe / Supervisor / Responsable'),'nivel_laboral'] = 4
avisos_detalles.loc[(avisos_detalles['nivel_laboral'] == 'Senior / Semi-Senior'),'nivel_laboral'] = 3
avisos_detalles.loc[(avisos_detalles['nivel_laboral'] == 'Junior'),'nivel_laboral'] = 2
avisos_detalles.loc[(avisos_detalles['nivel_laboral'] == 'Otro'),'nivel_laboral'] = 1

In [ ]:
avisos_detalles.sample(5)

In [ ]:
avisos_detalles.info()

## Merge del modelo con los postulantes y los detalles de los avisos

Cabe destacar que se realiza un inner join en ambos casos, para contar con registros con datos consistentes y sin valores nulos

In [ ]:
modelo = pd.merge(modelo, postulantes, on='idpostulante', how='inner')
modelo.sample(5)

In [ ]:
modelo.info()

In [ ]:
modelo = pd.merge(modelo, avisos_detalles, on='idaviso', how='inner')
modelo.sample(5)

In [ ]:
modelo.info()

Analizamos la información del modelo final que será utilizado por los algoritmos de Machine Learning.

Asignamos un 1 a aquellos valores NaN de la columna cant_vistas. Esto tiene dos propósitos:

- Mantener la coherencia. Una persona no puede postularse a un aviso sin visualizarlo.
- Evita valores nulos para los algoritmos de Machine Learning.

In [ ]:
#modelo.loc[(modelo['cant_vistas'].isnull()), 'cant_vistas'] = 1

Comprobamos que efectivamente no quedó ningún registro con cant_vistas con valor nulo.

In [ ]:
#modelo['cant_vistas'].isnull().sum()

Asignamos un 0 a aquellos valores NaN en las columnas 'edad', 'sexo', 'orden_estudio', 'tipo_de_trabajo' y 'nivel_laboral'. Al igual que con la columna 'cant_vistas', la finalidad de esto es evitar valores nulos para los algoritmos de Machine Learning.

In [ ]:
#modelo.loc[(modelo['orden_estudio'].isnull()), 'orden_estudio'] = 0
#modelo.loc[(modelo['tipo_de_trabajo'].isnull()), 'tipo_de_trabajo'] = 0
#modelo.loc[(modelo['nivel_laboral'].isnull()), 'nivel_laboral'] = 0
#modelo.loc[(modelo['sexo']=='NO_DECLARA'), 'sexo'] = 0
#modelo.loc[(modelo['sexo'].isnull()), 'sexo'] = 0
#modelo.loc[(modelo['edad'].isnull()), 'edad'] = 0
#modelo.loc[(modelo['nombre_zona'].isnull()), 'nombre_zona'] = 0

Reordenamos las columnas para facilitar el manejo del dataframe cuando apliquemos los algoritmos de ML

In [ ]:
#modelo = modelo[['idaviso','idpostulante','cant_vistas','edad','sexo',\
#                 'orden_estudio','titulo','descripcion','tipo_de_trabajo',\
#                 'nivel_laboral','nombre_zona','nombre_area','nombre_empresa',\
#                 'sepostulo']]

#modelo.sample(5)

In [ ]:
#modelo.info()

Analizamos la cantidad de postulaciones y no postulaciones con las que cuenta el dataframe.

In [ ]:
#modelo["sepostulo"].value_counts()

Considerando la gran cantidad de registros que tiene el dataframe, decidimos tomar una porción aleatoria y equitativa de postulaciones y no postulaciones.

In [ ]:
#no = modelo["sepostulo"] == 0
#si = modelo["sepostulo"] == 1

#modelo_no = modelo[no]
#modelo_si = modelo[si]

In [ ]:
#n_samples = 500000

#modelo_no = modelo_no.sample(n_samples)
#modelo_si = modelo_si.sample(n_samples)

#modelo_final = pd.merge(modelo_no, modelo_si, how='outer')

#modelo_final.sample(5)

In [ ]:
#modelo_final.info()

Exportamos el modelo final a un CSV file, para desentendernos de todas las operaciones costosas de dataframes a partir de este punto, y poder tener el set de entrenamiento ya listo para los algoritmos de Machine Learning.

In [ ]:
#modelo_final.to_csv("modelo.csv", encoding = "utf-8")